<a href="https://colab.research.google.com/github/Farhana-Najnin/Uncertainty-Aware-Demand-Forecaster/blob/main/Uncertainty_Aware_Demand_Forecaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#BLOCK 1 — Install dependencies (Streamlit + Torch + Ngrok + HF datasets)
!pip install -U datasets huggingface_hub pandas numpy scikit-learn matplotlib torch streamlit pyngrok





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 692.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 146.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

After installing packages:

👉 Runtime → Restart runtime


In [ ]:
#BLOCK 3 — Import + ngrok setup
from pyngrok import ngrok

# Optional: paste your ngrok token (recommended to avoid limits)
NGROK_TOKEN = "38NyKf7cg5H2mBodKTcyO2bmvTA_4yNoV7ahmLv8VYGvEYCu5"  # e.g. "2xY....."

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

print("ngrok ready")



ngrok ready


In [ ]:
#BLOCK 4 — Write model.py
%%writefile model.py
import torch
import torch.nn as nn

class GlobalLSTM(nn.Module):
    """
    Global multi-series LSTM with:
    - series embedding
    - dropout (used for MC Dropout uncertainty)
    """
    def __init__(self, n_series: int, emb_dim: int = 16, hidden: int = 64, dropout: float = 0.25):
        super().__init__()
        self.emb = nn.Embedding(n_series, emb_dim)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=1 + emb_dim, hidden_size=hidden, batch_first=True)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, sid: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        # sid: (B,)  x: (B, L)
        B, L = x.shape
        e = self.emb(sid)                      # (B, emb_dim)
        e_seq = e.unsqueeze(1).repeat(1, L, 1) # (B, L, emb_dim)

        x_seq = x.unsqueeze(-1)                # (B, L, 1)
        inp = torch.cat([x_seq, e_seq], dim=-1)

        out, _ = self.lstm(inp)
        last = out[:, -1, :]
        last = self.dropout(last)              # dropout active when model.train() during inference
        yhat = self.fc(last).squeeze(-1)
        return torch.clamp(yhat, min=0.0)



Writing model.py


In [ ]:
#BLOCK 5 — Write data_utils.py
%%writefile data_utils.py
import json
import numpy as np
import pandas as pd
from datasets import load_dataset

def load_store_sales_df() -> pd.DataFrame:
    ds = load_dataset("t4tiana/store-sales-time-series-forecasting")
    df = ds["train"].to_pandas()

    df["date"] = pd.to_datetime(df["date"])
    df = df.rename(columns={"store_nbr": "store_id", "family": "item_id", "sales": "y"})
    df["series_id"] = df["store_id"].astype(str) + "_" + df["item_id"].astype(str)

    return df.sort_values(["series_id", "date"]).reset_index(drop=True)

def train_val_split_last_h(df: pd.DataFrame, h: int = 28):
    train_rows, val_rows = [], []
    for sid, g in df.groupby("series_id"):
        if len(g) <= h:
            continue
        train_rows.append(g.iloc[:-h])
        val_rows.append(g.iloc[-h:])
    return pd.concat(train_rows).reset_index(drop=True), pd.concat(val_rows).reset_index(drop=True)

def fit_global_scaler(train_df: pd.DataFrame):
    y = train_df["y"].astype(float).values
    mu = float(np.mean(y))
    sigma = float(np.std(y) + 1e-8)
    return {"mean": mu, "std": sigma}

def apply_scaler(df: pd.DataFrame, scaler: dict) -> pd.DataFrame:
    out = df.copy()
    out["y_scaled"] = (out["y"].astype(float) - scaler["mean"]) / scaler["std"]
    return out

def save_scaler(scaler: dict, path: str):
    with open(path, "w") as f:
        json.dump(scaler, f)

def load_scaler(path: str) -> dict:
    with open(path, "r") as f:
        return json.load(f)

def build_series_index(df: pd.DataFrame):
    series_list = sorted(df["series_id"].unique().tolist())
    sid2idx = {s:i for i,s in enumerate(series_list)}
    idx2sid = {i:s for s,i in sid2idx.items()}
    return sid2idx, idx2sid

def make_windows(df_scaled: pd.DataFrame, sid2idx: dict, lookback: int = 56):
    """
    Safe window builder:
    - returns empty arrays if no windows exist (no crash)
    """
    S, X, Y = [], [], []

    for series_id, g in df_scaled.groupby("series_id"):
        g = g.sort_values("date")
        y = g["y_scaled"].values.astype(np.float32)

        if len(y) <= lookback:
            continue

        sid = sid2idx[series_id]
        for i in range(lookback, len(y)):
            S.append(sid)
            X.append(y[i-lookback:i])
            Y.append(y[i])

    if len(X) == 0:
        # Return empty but well-shaped arrays
        return (
            np.zeros((0,), dtype=np.int64),
            np.zeros((0, lookback), dtype=np.float32),
            np.zeros((0,), dtype=np.float32),
        )

    return (
        np.array(S, dtype=np.int64),
        np.stack(X).astype(np.float32),
        np.array(Y, dtype=np.float32),
    )


Writing data_utils.py


In [ ]:
#BLOCK 6 — Write inventory.py

%%writefile inventory.py
import numpy as np

def simulate_inventory_costs(
    demand: np.ndarray,
    forecast: np.ndarray,
    lead_time: int = 7,
    holding_cost: float = 1.0,
    stockout_cost: float = 10.0
):
    demand = demand.astype(float)
    forecast = forecast.astype(float)

    inventory = 0.0
    pipeline = [0.0] * lead_time

    holding_total = 0.0
    stockout_total = 0.0
    stockout_events = 0

    for d, f in zip(demand, forecast):
        # receive arriving order
        inventory += pipeline.pop(0)

        # meet demand
        if d > inventory:
            unmet = d - inventory
            stockout_total += unmet * stockout_cost
            inventory = 0.0
            stockout_events += 1
        else:
            inventory -= d

        # holding cost (end-of-day)
        holding_total += inventory * holding_cost

        # place order to cover lead time using forecast
        target = lead_time * max(f, 0.0)
        inv_position = inventory + sum(pipeline)
        order_qty = max(target - inv_position, 0.0)
        pipeline.append(order_qty)

    return {
        "total_cost": holding_total + stockout_total,
        "holding_cost": holding_total,
        "stockout_cost": stockout_total,
        "stockout_events": stockout_events,
        "days": len(demand),
    }


Writing inventory.py


In [ ]:
#BLOCK 7 — Write train_lstm.py (training + save artifacts)

%%writefile train_lstm.py
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from model import GlobalLSTM
from data_utils import (
    load_store_sales_df,
    train_val_split_last_h,
    fit_global_scaler,
    apply_scaler,
    save_scaler,
    build_series_index,
    make_windows
)

class WindowDataset(Dataset):
    def __init__(self, S, X, Y):
        self.S = torch.tensor(S, dtype=torch.long)
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.S[idx], self.X[idx], self.Y[idx]

def main():
    ART_DIR = "artifacts"
    os.makedirs(ART_DIR, exist_ok=True)

    LOOKBACK = 56
    H = 28
    MAX_SERIES = 250
    EPOCHS = 3
    BATCH = 512
    LR = 1e-3

    df = load_store_sales_df()

    # Limit series for speed/stability in Colab
    series_list = df["series_id"].unique()[:MAX_SERIES]
    df = df[df["series_id"].isin(series_list)].copy()

    train_df, _ = train_val_split_last_h(df, h=H)

    # Scale
    scaler = fit_global_scaler(train_df)
    save_scaler(scaler, os.path.join(ART_DIR, "scaler.json"))
    train_scaled = apply_scaler(train_df, scaler)

    # Series index
    sid2idx, _ = build_series_index(df)
    n_series = len(sid2idx)

    # Windows (TRAIN ONLY)
    S_tr, X_tr, Y_tr = make_windows(train_scaled, sid2idx, lookback=LOOKBACK)

    if len(Y_tr) == 0:
        raise RuntimeError(
            "No training windows were created. "
            "Try reducing LOOKBACK (e.g., 28) or increasing MAX_SERIES."
        )

    train_loader = DataLoader(WindowDataset(S_tr, X_tr, Y_tr), batch_size=BATCH, shuffle=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = GlobalLSTM(n_series=n_series, dropout=0.25).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = torch.nn.L1Loss()

    for epoch in range(EPOCHS):
        model.train()
        total = 0.0
        for sid, x, y in train_loader:
            sid, x, y = sid.to(device), x.to(device), y.to(device)
            opt.zero_grad()
            pred = model(sid, x)
            loss = loss_fn(pred, y)
            loss.backward()
            opt.step()
            total += loss.item()

        print(f"Epoch {epoch+1}/{EPOCHS} | train_loss={total/len(train_loader):.4f}")

    torch.save(
        {"state_dict": model.state_dict(), "n_series": n_series, "lookback": LOOKBACK, "max_series": MAX_SERIES},
        os.path.join(ART_DIR, "lstm_mc_dropout.pt")
    )

    print("Saved artifacts:")
    print("- artifacts/lstm_mc_dropout.pt")
    print("- artifacts/scaler.json")

if __name__ == "__main__":
    main()


Writing train_lstm.py


In [ ]:
#BLOCK 8 — Write app.py (Streamlit + MC Dropout + Inventory simulation)(AUTO-TRAIN FIX)
%%writefile app.py
import os
import numpy as np
import pandas as pd
import streamlit as st
import torch
import subprocess
import sys

from model import GlobalLSTM
from data_utils import load_store_sales_df, load_scaler, apply_scaler, build_series_index
from inventory import simulate_inventory_costs

st.set_page_config(page_title="Demand Forecasting Dashboard", layout="wide")

# ---------- PATHS (absolute, Colab-safe) ----------
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
ART_DIR = os.path.join(BASE_DIR, "artifacts")
SCALER_PATH = os.path.join(ART_DIR, "scaler.json")
MODEL_PATH = os.path.join(ART_DIR, "lstm_mc_dropout.pt")


def ensure_artifacts_exist():
    """
    If artifacts are missing, train the model ONCE from inside the app.
    This prevents FileNotFoundError when opening public URL.
    """
    os.makedirs(ART_DIR, exist_ok=True)

    if os.path.exists(SCALER_PATH) and os.path.exists(MODEL_PATH):
        return

    st.warning("Model artifacts not found. Training now (one-time)...")

    # Run training script
    result = subprocess.run(
        [sys.executable, "train_lstm.py"],
        capture_output=True,
        text=True
    )

    # Show logs if something fails
    if result.returncode != 0:
        st.error("Training failed. Logs below:")
        st.code(result.stdout + "\n" + result.stderr)
        st.stop()

    # Double check
    if not (os.path.exists(SCALER_PATH) and os.path.exists(MODEL_PATH)):
        st.error("Training completed but artifacts still missing. Check path issues.")
        st.stop()

    st.success("Training complete. Artifacts created ✅")


@st.cache_data
def load_data(max_series=250):
    df = load_store_sales_df()
    series = df["series_id"].unique()[:max_series]
    df = df[df["series_id"].isin(series)].copy()
    df = df.sort_values(["series_id", "date"]).reset_index(drop=True)
    return df


@st.cache_resource
def load_artifacts():
    ensure_artifacts_exist()

    scaler = load_scaler(SCALER_PATH)
    ckpt = torch.load(MODEL_PATH, map_location="cpu")

    model = GlobalLSTM(n_series=ckpt["n_series"], dropout=0.25)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()

    return scaler, ckpt, model


def inverse_scale(arr_scaled, scaler):
    return arr_scaled * scaler["std"] + scaler["mean"]


def mc_forecast_quantiles(model, sid_idx, history_scaled, horizon=28, mc_samples=30, q_list=(50, 90)):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    lookback = len(history_scaled)
    x = history_scaled.astype(np.float32).copy()

    preds_q = {q: [] for q in q_list}

    for _ in range(horizon):
        sid = torch.tensor([sid_idx], dtype=torch.long).to(device)
        x_t = torch.tensor(x.reshape(1, lookback), dtype=torch.float32).to(device)

        # enable dropout for MC sampling
        model.train()

        samples = []
        with torch.no_grad():
            for _m in range(mc_samples):
                yhat = model(sid, x_t).cpu().numpy()[0]
                samples.append(yhat)

        samples = np.array(samples)
        for q in q_list:
            preds_q[q].append(np.percentile(samples, q))

        # feed back P50
        x = np.roll(x, -1)
        x[-1] = preds_q[50][-1]

    model.eval()
    return {q: np.array(vals) for q, vals in preds_q.items()}


# ---------- UI ----------
st.title("Demand Forecasting (Global LSTM + MC Dropout) + Inventory Simulation")

with st.sidebar:
    st.header("Controls")
    max_series = st.slider("Max series (speed)", 50, 500, 250, step=50)
    horizon = st.slider("Forecast horizon (days)", 7, 60, 28, step=7)
    lookback = st.slider("Lookback window (days)", 28, 90, 56, step=7)
    mc_samples = st.slider("MC Dropout samples", 10, 100, 30, step=10)

    st.subheader("Inventory policy")
    lead_time = st.slider("Lead time (days)", 1, 21, 7, step=1)
    holding_cost = st.number_input("Holding cost (per unit)", value=1.0, min_value=0.0, step=0.5)
    stockout_cost = st.number_input("Stockout cost (per unit unmet)", value=10.0, min_value=0.0, step=1.0)
    service_q = st.selectbox("Forecast used for ordering", options=[50, 90], index=1)

df = load_data(max_series=max_series)
scaler, ckpt, model = load_artifacts()
sid2idx, _ = build_series_index(df)

series_choices = df["series_id"].unique().tolist()
series_id = st.selectbox("Choose a store-family series", series_choices, index=0)

g = df[df["series_id"] == series_id].sort_values("date").copy()
if len(g) < lookback + horizon:
    st.warning("Series too short for lookback+horizon. Pick another or reduce sliders.")
    st.stop()

g_scaled = apply_scaler(g, scaler)

history = g_scaled["y_scaled"].values[-lookback:]
actual_window = g["y"].values[-horizon:]
dates_window = g["date"].values[-horizon:]

sid_idx = sid2idx[series_id]

qs = mc_forecast_quantiles(
    model=model,
    sid_idx=sid_idx,
    history_scaled=history,
    horizon=horizon,
    mc_samples=mc_samples,
    q_list=(50, 90),
)

p50 = inverse_scale(qs[50], scaler)
p90 = inverse_scale(qs[90], scaler)

order_forecast = p90 if service_q == 90 else p50

inv_policy = simulate_inventory_costs(actual_window, order_forecast, lead_time, holding_cost, stockout_cost)
inv_p50 = simulate_inventory_costs(actual_window, p50, lead_time, holding_cost, stockout_cost)
inv_p90 = simulate_inventory_costs(actual_window, p90, lead_time, holding_cost, stockout_cost)

c1, c2 = st.columns([2, 1])

with c1:
    st.subheader("Forecast Plot (P50 / P90)")
    plot_df = pd.DataFrame({"date": dates_window, "actual": actual_window, "p50": p50, "p90": p90}).set_index("date")
    st.line_chart(plot_df)

with c2:
    st.subheader("Inventory KPIs")
    st.write(f"**Ordering policy:** P{service_q}")
    st.metric("Total Cost", f"{inv_policy['total_cost']:.2f}")
    st.metric("Holding Cost", f"{inv_policy['holding_cost']:.2f}")
    st.metric("Stockout Cost", f"{inv_policy['stockout_cost']:.2f}")
    st.metric("Stockout Events", f"{inv_policy['stockout_events']} / {inv_policy['days']}")

    st.subheader("Policy Comparison")
    st.write("P50:", inv_p50)
    st.write("P90:", inv_p90)

st.subheader("Recent Data")
st.dataframe(g[["date", "store_id", "item_id", "y"]].tail(25), use_container_width=True)


Writing app.py


In [ ]:
#BLOCK 9 — Train the model (creates artifacts/)
!python train_lstm.py



README.md: 3.24kB [00:00, 5.21MB/s]
Repo card metadata block was not found. Setting CardData to empty.
train.csv: 100% 122M/122M [00:02<00:00, 60.5MB/s] 
test.csv: 1.02MB [00:00, 153MB/s]
Generating train split: 3000888 examples [00:02, 1015509.21 examples/s]
Generating test split: 28512 examples [00:00, 968465.00 examples/s]
Epoch 1/3 | train_loss=0.3231
Epoch 2/3 | train_loss=0.3049
Epoch 3/3 | train_loss=0.3013
Saved artifacts:
- artifacts/lstm_mc_dropout.pt
- artifacts/scaler.json


In [ ]:
#BLOCK 10 — Run Streamlit + expose with ngrok (public URL)
import subprocess,time
from pyngrok import ngrok

try: ngrok.kill()
except: pass

subprocess.Popen([
 "streamlit","run","app.py",
 "--server.port=8501",
 "--server.address=0.0.0.0",
 "--server.enableCORS=false",
 "--server.enableXsrfProtection=false"
])

time.sleep(3)
print("PUBLIC URL:",ngrok.connect(8501,"http"))


PUBLIC URL: NgrokTunnel: "https://endurable-thwartedly-somer.ngrok-free.dev" -> "http://localhost:8501"
